In [1]:
!pip install pyspark==3.3.1


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession\
    .builder\
    .appName('twitter_transformation')\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


24/03/05 10:38:50 WARN Utils: Your hostname, BITL0401 resolves to a loopback address: 127.0.1.1; using 172.27.38.146 instead (on interface eth0)
24/03/05 10:38:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/05 10:38:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.json('../../datalake/twitter_datascience')

In [6]:
df.show()

+--------------------+--------------------+------------+
|                data|            includes|extract_date|
+--------------------+--------------------+------------+
|[{30, 13, 2024-03...|{[{2024-03-02T07:...|  2024-03-02|
|[{64, 90, 2024-03...|{[{2024-03-03T05:...|  2024-03-03|
|[{74, 48, 2024-03...|{[{2024-03-04T11:...|  2024-03-04|
+--------------------+--------------------+------------+



In [7]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [8]:
from pyspark.sql import functions as f

In [9]:
df.select(f.explode('data')).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [10]:
df.select(f.explode('data')).show()

+--------------------+
|                 col|
+--------------------+
|{30, 13, 2024-03-...|
|{78, 21, 2024-03-...|
|{20, 37, 2024-03-...|
|{44, 40, 2024-03-...|
|{48, 20, 2024-03-...|
|{5, 76, 2024-03-0...|
|{39, 18, 2024-03-...|
|{62, 29, 2024-03-...|
|{72, 41, 2024-03-...|
|{98, 55, 2024-03-...|
|{64, 90, 2024-03-...|
|{56, 59, 2024-03-...|
|{12, 57, 2024-03-...|
|{73, 75, 2024-03-...|
|{86, 13, 2024-03-...|
|{27, 50, 2024-03-...|
|{1, 88, 2024-03-0...|
|{22, 40, 2024-03-...|
|{50, 28, 2024-03-...|
|{53, 87, 2024-03-...|
+--------------------+
only showing top 20 rows



In [11]:
df.select(f.explode("data").alias("tweets"))\
.select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [12]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
.select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [13]:
tweet_df.show(5)

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|       30|             13|2024-03-02T08:12:...| 35|        99|         22|         12|           27|Tweet fictício cr...|
|       78|             21|2024-03-02T21:31:...|100|        15|         63|          5|           17|Tweet fictício ge...|
|       20|             37|2024-03-02T23:51:...| 52|        87|         11|         60|           18|Este é um tweet f...|
|       44|             40|2024-03-02T12:03:...| 87|        45|         32|         55|           52|Um terceiro tweet...|
|       48|             20|2024-03-02T04:01:...| 12|        82|          1|          9|           51|Tweet fictício ge...|
+---------+-----